In [1]:
# Example of Logistic Regression
# Michael Timbes

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [3]:
def normalize(x):
    """
    Normalizes an input vector.
    """
    mu = np.mean(x)
    st = np.std(x)
    for i in range(len(x)):
        x[i] = ((x[i]-mu)/st)
    return x

In [4]:
def mklabels(Y_):
    """
    Input is a Y vector (1-D) and makes two label classes [1,0] true [0,1] false.
    Returns array of the classes.
    """
    num = len(Y_)
    Y_temp = np.zeros([num,2])
    for i in range(num):
        if (Y_[i] == 1):
            Y_temp[i,0] = 1 # Make True Class
        if (Y_[i] == 0):
            Y_temp[i,1] = 1 # Make False Class
    return np.asarray(Y_temp)

In [5]:
# Data Preparation
data_frame =pd.read_csv('HeartData.csv', sep=',')
data_frame = np.asarray(data_frame)

# Assign Training Data to Input vectors
X_train = data_frame[0:50,1] # Ages 
Y_train = data_frame[0:50,2] # Heart Disease(1) or not Heart Disease(0)

# Assign Test Data
X_test = data_frame[50:100,1] # Ages
Y_test = data_frame[50:100,2] # Heart Disease(1) or not Heart Disease(0)
# Normalize Along X
X_train = normalize(X_train)
num_tr = len(X_train)

# Make Y a 2-D Vector for true class and false class
Y_temp = np.zeros([num_tr,2])

# Ensure One-Hot Encoding
Y_train = mklabels(Y_train)
Y_test = mklabels(Y_test)
print("Number of trainig input:",num_tr)

Number of trainig input: 50


In [6]:
# Model
alpha = 0.5
X = tf.placeholder(tf.float32,[None,1]) # Conection for X vals
Y = tf.placeholder(tf.float32,[None,2]) # Connection for Y vals

# Design Matrix

W = tf.Variable(tf.zeros([1,2])) # Two Variables for classification
b = tf.Variable(tf.zeros([2])) # 
#h = tf.matmul(X,W) + b
h = tf.nn.sigmoid(tf.matmul(X,W) + b)
# Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=h))

# Optimization
opt_step = tf.train.GradientDescentOptimizer(alpha).minimize(cost)

# Initialization
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
x_batch = np.ones((1, 1))
y_batch = np.ones((1, 2))

In [7]:
# Training 
for i in range(500):
    r = np.random.permutation(range(num_tr))
    X_train,Y_train = X_train[r], Y_train[r]
    for j in range(num_tr):
        x_batch[0,:] = X_train[j]
        y_batch[0,:] = Y_train[j]
        session.run(opt_step,feed_dict={X:x_batch,Y:y_batch})

In [8]:
# Verification
for j in range(num_tr):
        x_batch[0,:] = X_train[j]
        y_batch[0,:] = Y_train[j]
        b_reg, W_reg, loss_result = session.run([b,W,cost],feed_dict={X:x_batch,Y:y_batch})
print("W= ",W_reg)
print("B= ",b_reg)
print("Total Loss: ", loss_result)

W=  [[ 0.22967467 -0.22967562]]
B=  [-7.00953913  7.00954771]
Total Loss:  1.31221


In [9]:
# Testing
accuracy_result = []
for j in range(num_tr):
        x_batch[0,:] = X_test[j]
        y_batch[0,:] = Y_test[j]
        correct_prediction = tf.equal(tf.argmax(h,1), tf.argmax(Y,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        accuracy_result.append(accuracy.eval(feed_dict={X:x_batch,Y:y_batch}))
accuracy_result = np.asarray(accuracy_result)
print(np.mean(accuracy_result))
print(accuracy_result[0:9])
print(Y_test[0:9])

0.68
[ 1.  1.  0.  1.  0.  1.  0.  0.  1.]
[[ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [10]:
for i in range(500):
    r = np.random.permutation(range(num_tr))
    X_test,Y_test = X_test[r], Y_test[r]
    for j in range(num_tr):
        x_batch[0,:] = X_train[j]
        y_batch[0,:] = Y_train[j]
        session.run(opt_step,feed_dict={X:x_batch,Y:y_batch})